In [1]:
import sys
sys.path.append("./CVAE")

In [2]:
!pip install pyro-ppl
from main import main
import argparse
import itertools
import pandas as pd
import shutil
import os
import glob
import os

In [3]:
# Define the hyperparameter search space
lr = [1e-4, 1e-3]
num_epochs = [1,50, 101]
num_particl = [20, 10]

# Iterate over all combinations of hyperparameters
for lr, num_epochs, num_particles in itertools.product(lr, num_epochs, num_particl):
    # Set up arguments
    args = argparse.Namespace(
        num_quadrant_inputs=[1, 2, 3],
        num_epochs=num_epochs,
        early_stop_patience=3,
        learning_rate=lr,
        cuda=True,
        num_images=10,
        num_samples=10,
        num_particles=num_particles,
        dataset='cifar10',
        random_mask=False,
        z_dim=200,
        hidden_1=500,
        hidden_2=500,
        allow_baseline_reuse=False,
        use_conv=False
    )

    # Log the current configuration
    print(f"Running with lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}")

    # Run the main function
    try:
        main(args)
    except Exception as e:
        print(f"Error with configuration lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}: {e}")
        continue

    # Save the results
    if os.path.exists("results.csv"):
        output_file = f"tuning_results/results_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.csv"
        shutil.copy("results.csv", output_file)
        print(f"Results saved to {output_file}")
    else:
        print(f"No 'results.csv' file generated for lr={lr}, num_epochs={num_epochs}, num_particles={num_particles}.")
    # Backup important files before starting
    if os.path.exists("cvae_plot_q1.png"):
        output_file = f"tuning_results/cvae_ploth_q1_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.png"
        shutil.copy("cvae_plot_q1.png", output_file)
        print("Backup created for 'cvae_plot_q1.png'.")
    if os.path.exists("cvae_plot_q2.png"):
        output_file = f"tuning_results/cvae_ploth_q2_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.png"
        shutil.copy("cvae_plot_q2.png", output_file)
        print("Backup created for 'cvae_plot_q2.png'.")
    if os.path.exists("cvae_plot_q3.png"):
        output_file = f"tuning_results/cvae_ploth_q3_lr_{lr}_epochs_{num_epochs}_particles_{num_particles}.png"
        shutil.copy("cvae_plot_q3.png", output_file)
        print("Backup created for 'cvae_plot_q3.png'.")


Running with lr=0.0001, num_epochs=1, num_particles=20
Device:  cuda:0
Training with 1 quadrant as input...
Files already downloaded and verified
Files already downloaded and verified


NN Epoch 0 train    :  41%|████      | 161/391 [00:03<00:05, 42.84it/s, early_stop_count=0, loss=1563.01]


KeyboardInterrupt: 

In [ ]:
# Combine all results for analysis
result_files = glob.glob("tuning_results/*.csv")
if result_files:
    combined_results = []

    for file in result_files:
        # Read each result file
        df = pd.read_csv(file)

        # Rename the first column if necessary
        if 'Unnamed: 0' in df.columns:
            df.rename(columns={'Unnamed: 0': 'Method'}, inplace=True)

        print(df)

        # Extract performance gaps and compute the aggregate gap
        performance_gap_row = df.loc[df['Method'] == 'Performance gap']
        aggregate_gap = performance_gap_row[['1 quadrant', '2 quadrants', '3 quadrants']].sum(axis=1).values[0]

        # Store results with the file name and aggregate gap
        combined_results.append({
            "file": file,
            "aggregate_gap": aggregate_gap
        })

    # Convert to a DataFrame for sorting and analysis
    combined_results_df = pd.DataFrame(combined_results)

    # Sort by aggregate gap in descending order (largest gap first)
    combined_results_df = combined_results_df.sort_values(by="aggregate_gap", ascending=False)

    # Save the combined results to a file
    combined_results_df.to_csv("tuning_results/combined_results.csv", index=False)
    print("All results combined and saved to 'tuning_results/combined_results.csv'.")

    # Display the best configuration
    print("Best configuration:")
    best_file = combined_results_df.iloc[0]['file']
    print(f"File: {best_file}, Aggregate Performance Gap: {combined_results_df.iloc[0]['aggregate_gap']}")

    # Optionally, load and display the details of the best result
    best_result = pd.read_csv(best_file)
    print("\nDetails of the best configuration:")
    print(best_result)
else:
    print("No results to combine.")
